In [1]:
!pwd

/home/yutankim/Desktop/EfficientDet_TensorFlow2


In [9]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader

def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

#모델 구조
model = EfficientDet()
print_model_summary(model)

#weight 불러오기
load_weights_from_epoch = Config.load_weights_from_epoch_quan
model.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))
#model.load_weights(filepath=Config.save_model_dir+"saved_model")

Model: "efficient_det_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficient_net_2 (EfficientNe multiple                  6771296   
_________________________________________________________________
bi_fpn_2 (BiFPN)             multiple                  129126    
_________________________________________________________________
box_class_predict_2 (BoxClas multiple                  249069    
Total params: 7,149,491
Trainable params: 7,106,515
Non-trainable params: 42,976
_________________________________________________________________


In [6]:
import numpy  as np
from PIL import Image
import os

path = './data/fire_smoke/train/JPEGImages'
count = 0
image_list = []
for item in os.listdir(path):
    imgpath = path +'/' + item
    img = Image.open(imgpath)
    arr = np.array(img)
    image_list.append(arr)
    #img.save(str(count) +  "output.png")
    count = count +1
print(np.array(image_list).shape)

for image in tf.data.Dataset.from_tensor_slices(image_list).batch(32).take(10):
    image = tf.image.resize(image, (512,512))
    print(image.shape)

(516, 480, 640, 3)


In [10]:
# 여기 부분, 기존 코드의 data_preprocesssing 과 동일하게 맞추기
# 배치사이즈 ( 32 ) 에 동일하게 맞춰서 넣어줘야한다.
# def representative_data_gen():
#     dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
#     for i in range(100):
#         image = next(iter(dataset_list))
#         image = tf.io.read_file(image)
#         image = tf.io.decode_jpeg(image, channels=3)
#         image = tf.image.resize(image, (512,512))
#         #image = tf.cast(image / 255., tf.float32)
#         image = tf.expand_dims(image, 0)
#         print(image.shape)
# # Model has only one input so each data point has one element
#         yield [image]
    
# def representative_data_gen():
#     dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
#     image_data = []
#     for dir in dataset_list:
        
    
#     for j in range(10):
#         image_temp = [ ]
#         for temp_dir in dataset_list.take(32):
#             image = tf.io.read_file(temp_dir)
#             image = tf.io.decode_jpeg(image, channels=3)
#             image = tf.image.resize(image, (512,512))
#             #image = tf.cast(image / 255., tf.float32)
#             image = tf.expand_dims(image, 0)
#             image_temp.append(image)
#     # Model has only one input so each data point has one element
#         yield [image_temp]
        
def representative_data_gen():
    path = './data/fire_smoke/train/JPEGImages'
    count = 0
    image_list = []
    for item in os.listdir(path):
        imgpath = path +'/' + item
        img = Image.open(imgpath)
        arr = np.array(img)
        image_list.append(arr)
        #img.save(str(count) +  "output.png")
        count = count +1
    print(np.array(image_list).shape)
    
    for image in tf.data.Dataset.from_tensor_slices(image_list).batch(32).take(10):
        image = tf.image.resize(image, (512,512))
        # 데이터 전처리 하건거.. ??? 
        #리사이즈만 !! 
        yield [image]
            
#     #### 수정좀 해보기
# def representative_data_gen():
#   dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
#   for i in range(100):
#     image = next(iter(dataset_list))
#     image = tf.io.read_file(image)
#     image = tf.io.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (512,512))
#     # image = tf.cast(image / 255., tf.float32)
#     image = tf.expand_dims(image, 0)
# # Model has only one input so each data point has one element
#     yield [image]

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model_quant = converter.convert()

2021-10-26 16:08:10.510316: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-10-26 16:08:10.510464: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-10-26 16:08:10.513389: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:af:00.0, compute capability: 7.5
2021-10-26 16:08:10.695039: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.02ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.

2021-10-26 16:08:13.319046: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-10-26 16:08:13.319126: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuff

(516, 480, 640, 3)


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0
2021-10-26 16:19:49.822664: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1899] Estimated count of arithmetic ops: 235.640 G  ops, equivalently 117.820 G  MACs


In [12]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.float32'>


In [13]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

name : input_1
index : 0
shape : [ 32 512 512   3]
shape_signature : [ 32 512 512   3]
dtype : <class 'numpy.uint8'>
quantization : (1.0, 0)
quantization_parameters : {'scales': array([1.], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_14/mb_conv_32/se_block_32/global_average_pooling2d_32/Mean/reduction_indices
index : 1
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/bi_fpn_2/bi_fpn_module_4/up_sampling2d_16/mul
index : 2
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

na

      dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_17/mb_conv_37/batch_normalization_168/FusedBatchNormV3;efficient_det_2/efficient_net_2/sequential_17/mb_conv_37/se_block_37/conv2d_179/Conv2D;efficient_det_2/efficient_net_2/sequential_17/mb_conv_37/conv2d_177/Conv2D
index : 65
shape : [240]
shape_signature : [240]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.587174  , 0.75179446, 0.7193323 , 0.65631413, 0.7837086 ,
       0.6049691 , 0.6858042 , 0.6560149 , 0.69150275, 0.791329  ,
       0.71316546, 0.7065046 , 0.75314313, 0.83312404, 0.6905037 ,
       0.7656604 , 0.69729567, 0.73363966, 0.7008909 , 0.63976485,
       0.68058246, 0.893196  , 0.6782943 , 0.7110213 , 0.6537293 ,
       0.69926846, 0.6721173 , 0.6884711 , 0.7847127 , 0.6565268 ,
       0.64625925, 0.71977675, 0.6488726 , 0.7246152 , 0.701418  ,
       0.78023875, 0.7502658 , 0.68732285, 0.9174522

sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_17/mb_conv_38/batch_normalization_172/FusedBatchNormV3;efficient_det_2/efficient_net_2/sequential_17/mb_conv_38/depthwise_conv2d_38/depthwise;efficient_det_2/efficient_net_2/sequential_18/mb_conv_40/se_block_40/conv2d_191/Conv2D
index : 76
shape : [  1   3   3 480]
shape_signature : [  1   3   3 480]
dtype : <class 'numpy.int8'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.00448914, 0.0065648 , 0.0038183 , 0.00430209, 0.00424285,
       0.00563211, 0.00515513, 0.00438356, 0.00618545, 0.00683269,
       0.00625742, 0.00579265, 0.00409846, 0.00470175, 0.00477838,
       0.00303105, 0.00580545, 0.00818953, 0.00579502, 0.00561243,
       0.00806933, 0.00765994, 0.00595341, 0.00566872, 0.00410246,
       0.00492598, 0.00428568, 0.00623532, 0.0063507 , 0.00619981,
       0.00836466, 0.00276431, 0.00568101, 0.00499448, 0.00650565,
       0.00621331, 0.00484299, 0.00537655, 0.00317176, 0.005016


name : efficient_det_2/efficient_net_2/sequential_18/mb_conv_40/se_block_40/conv2d_190/Conv2D
index : 102
shape : [120   1   1 480]
shape_signature : [120   1   1 480]
dtype : <class 'numpy.int8'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.00080286, 0.00081049, 0.00083356, 0.0008021 , 0.00079163,
       0.00079526, 0.00080879, 0.00080224, 0.0007931 , 0.00082179,
       0.000799  , 0.00080051, 0.00080484, 0.00081391, 0.00079571,
       0.00082225, 0.00079786, 0.00084109, 0.00081477, 0.00082179,
       0.00080017, 0.00082328, 0.00080936, 0.00081847, 0.00081271,
       0.00079738, 0.0007982 , 0.00079174, 0.0008457 , 0.00082215,
       0.00080005, 0.00080411, 0.0008218 , 0.00079277, 0.00080296,
       0.00079532, 0.00080319, 0.00082439, 0.00078794, 0.00078519,
       0.00081228, 0.00081693, 0.00079908, 0.0008259 , 0.00082222,
       0.00083212, 0.00078862, 0.00081877, 0.00079793, 0.00080077,
       0.00079767, 0.00080409, 0.00080796, 0.00081628, 0.00079261,
    

quantization_parameters : {'scales': array([0.00366876, 0.0034561 , 0.00285929, 0.00249493, 0.00488477,
       0.00296242, 0.00307619, 0.00398414, 0.00237322, 0.00338299,
       0.00333935, 0.00282894, 0.00295212, 0.00256872, 0.00380745,
       0.00388754, 0.00421355, 0.00356144, 0.00471634, 0.00484345,
       0.00483865, 0.00394379, 0.00474153, 0.00309683, 0.00275395,
       0.00253018, 0.00222138, 0.00335035, 0.00304093, 0.00356076,
       0.0032865 , 0.00385475, 0.00382492, 0.00226974, 0.0033725 ,
       0.00359168, 0.00401783, 0.00389188, 0.00397043, 0.00545903,
       0.00310915, 0.0023405 , 0.00365686, 0.00443216, 0.0024897 ,
       0.00424801, 0.00415107, 0.00301472, 0.00332409, 0.0038193 ,
       0.00300639, 0.00295069, 0.00306417, 0.00316201, 0.00318881,
       0.00288664, 0.00357591, 0.00405899, 0.00222773, 0.00279804,
       0.00379365, 0.00448672, 0.00326887, 0.00387755, 0.00231596,
       0.00383229, 0.00390005, 0.00414032, 0.00248565, 0.00419449,
       0.00288646, 0.0033

...

quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_17/mb_conv_37/se_block_37/mul_1
index : 421
shape : [ 32  32  32 240]
shape_signature : [ 32  32  32 240]
dtype : <class 'numpy.int8'>
quantization : (496.07275390625, -11)
quantization_parameters : {'scales': array([496.07275], dtype=float32), 'zero_points': array([-11], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_17/mb_conv_37/Sigmoid_1
index : 422
shape : [ 32  32  32 240]
shape_signature : [ 32  32  32 240]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_17/mb_conv_37/

name : efficient_det_2/efficient_net_2/sequential_18/mb_conv_41/se_block_41/Sigmoid
index : 480
shape : [ 32   1   1 168]
shape_signature : [ 32   1   1 168]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_18/mb_conv_41/se_block_41/mul
index : 481
shape : [ 32   1   1 168]
shape_signature : [ 32   1   1 168]
dtype : <class 'numpy.int8'>
quantization : (890.052734375, -128)
quantization_parameters : {'scales': array([890.05273], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/efficient_net_2/sequential_18/mb_conv_41/se_block_41/conv2d_195/BiasAdd;efficient_det_2/efficient_net_2/sequential_19/mb_conv_43/se_block_43/conv2d_203/Conv2D;efficient_det_2/efficient_net_2/sequential

quantization_parameters : {'scales': array([7257.4697], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/bi_fpn_2/bi_fpn_module_4/weighted_feature_fusion_32/mul_1
index : 591
shape : [32  8  8 64]
shape_signature : [32  8  8 64]
dtype : <class 'numpy.int8'>
quantization : (3763.056884765625, -128)
quantization_parameters : {'scales': array([3763.057], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/bi_fpn_2/bi_fpn_module_4/weighted_feature_fusion_32/Sum/input
index : 592
shape : [ 2 32  8  8 64]
shape_signature : [ 2 32  8  8 64]
dtype : <class 'numpy.int8'>
quantization : (3763.056884765625, -128)
quantization_parameters : {'scales': array([3763.057], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/bi_fpn_2/bi_fpn_module_4/weighted_

quantization_parameters : {'scales': array([7033.8765], dtype=float32), 'zero_points': array([12], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/bi_fpn_2/bi_fpn_module_5/weighted_feature_fusion_44/separable_conv_norm_act_44/Sigmoid
index : 728
shape : [32 32 32 64]
shape_signature : [32 32 32 64]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/bi_fpn_2/bi_fpn_module_5/weighted_feature_fusion_44/separable_conv_norm_act_44/mul
index : 729
shape : [32 32 32 64]
shape_signature : [32 32 32 64]
dtype : <class 'numpy.int8'>
quantization : (3173.77490234375, -128)
quantization_parameters : {'scales': array([3173.775], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_

quantization_parameters : {'scales': array([804.20355], dtype=float32), 'zero_points': array([-6], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_2/bi_fpn_2/bi_fpn_module_5/weighted_feature_fusion_45/separable_conv_norm_act_45/batch_normalization_219/FusedBatchNormV3;efficient_det_2/bi_fpn_2/bi_fpn_module_5/weighted_feature_fusion_45/separable_conv_norm_act_45/separable_conv2d_45/BiasAdd/ReadVariableOp/resource;efficient_det_2/bi_fpn_2/bi_fpn_module_5/weighted_feature_fusion_45/separable_conv_norm_act_45/separable_conv2d_45/BiasAdd;efficient_det_2/box_class_predict_2/conv_norm_act_32/conv2d_231/Conv2D_4;efficient_det_2/bi_fpn_2/bi_fpn_module_5/weighted_feature_fusion_45/separable_conv_norm_act_45/separable_conv2d_45/separable_conv2d
index : 760
shape : [32 16 16 64]
shape_signature : [32 16 16 64]
dtype : <class 'numpy.int8'>
quantization : (7750.92822265625, 1)
quantization_parameters : {'scales': array([7750.928], dtype=float32), 'zero_points':

In [14]:
with open('./tflite_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [15]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print(output_details)

{'name': 'input_1', 'index': 0, 'shape': array([ 32, 512, 512,   3], dtype=int32), 'shape_signature': array([ 32, 512, 512,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (1.0, 0), 'quantization_parameters': {'scales': array([1.], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'Identity', 'index': 856, 'shape': array([   32, 49104,     5], dtype=int32), 'shape_signature': array([   32, 49104,     5], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [16]:
model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    return interpreter

def prediction(model, data):
    interpreter = model
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    input_data = np.array(data, dtype = np.uint8)
    interpreter.set_tensor(input_details['index'], input_data)
    interpreter.invoke()
    output= interpreter.get_tensor(output_details['index'])
    return output

In [17]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')

image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
# image = tf.cast(image / 255., tf.float32)
image = tf.expand_dims(image, 0)
print(image.shape)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: ./data/fire_smoke/train/JPEGImages\\*'

In [ ]:
import numpy as np

model = load_model(model_path)
output = prediction(model, image)

In [ ]:
print(output.shape)
print(output)